In [140]:
import sys 
sys.path.append('../')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from NN import network, activation_functions, loss_functions
from NN import ANN
X = np.array([[1,2],[3,4]])


## First Code running

In [141]:
# Random Init
teste = ANN.ANN("sigmoid")
teste.initialize_random_weights(2, [3], 2)
teste.show_weights()
teste.show_setup()
Y, aY = teste.foward_propagation(X)
print("Y\n",Y)
print("\na(Y)\n",aY)

Hidden Layer  1
[[0.27605372 0.1906593  0.30017076]
 [0.88279538 0.94562778 0.34077477]
 [0.64280095 0.65985464 0.67032014]] 

Output Layer 
[[0.59598795 0.87561001]
 [0.26656552 0.01072513]
 [0.45661502 0.13918876]
 [0.96476747 0.11160428]] 

 Input size:  2
 Number of hidden layers:  1
 Number of perceptrons at each layer: 
 HL 1: 3
 Number of classes: 2 

Y
 [[2.10969738 1.1117549 ]
 [2.2639843  1.13460472]]

a(Y)
 [[0.89184215 0.75245614]
 [0.90584999 0.75668768]]


## Comparing with the new Implementation by using the same weights

In [142]:
from importlib import reload
reload(network)
I = np.array([[0.05, 0.10]])
W = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.35, label="H1")
net, out =  h1.feed_forward(I)
print(net)
print(out)

H1 (1, 2) (2, 2)
net [[0.3775 0.3925]]
out [[0.59326999 0.59688438]]
[[0.3775 0.3925]]
[[0.59326999 0.59688438]]


In [143]:
from importlib import reload
reload(network)
I = np.array([[0.59326999, 0.59688438]])
W = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o1 = network.Layer(2, 2, 'sigmoid', weights=W, bias=0.6, label="H1")
net, out =  o1.feed_forward(I)
print(net)
print(out)

H1 (1, 2) (2, 2)
net [[1.10590597 1.2249214 ]]
out [[0.75136507 0.77292847]]
[[1.10590597 1.2249214 ]]
[[0.75136507 0.77292847]]


In [144]:
from importlib import reload
reload(network)
model = network.NN(loss='mse')
X = np.array([[0.05, 0.10]])
Wh1 = np.array([[0.15, 0.2], [0.25, 0.3]]).T
h1 = network.Layer(2, 2, 'sigmoid', weights=Wh1, bias=0.35, label="H1")
Wo1 = np.array([[0.4, 0.45], [0.50, 0.55]]).T
o1 = network.Layer(2, 2, 'sigmoid', weights=Wo1, bias=0.6, label="H1")
model.add_layer(h1)
model.add_layer(o1)
model.show_weights()

Y_, aY_ = model.feed_forward(X)
Y, aY = ([1.10590597, 1.2249214 ], [0.75136507, 0.77292847])
print("Y\n",Y_)
print("\na(Y)\n",aY_)

assert((Y - Y_).sum() < np.finfo(np.float32).eps)
assert((aY - aY_).sum() < np.finfo(np.float32).eps)


Model Weights
-------------------------------
H1      (input=2, neurons=2, activation=sigmoid)
[[0.15 0.25]
 [0.2  0.3 ]]
H1      (input=2, neurons=2, activation=sigmoid)
[[0.4  0.5 ]
 [0.45 0.55]]
-------------------------------
H1 (1, 2) (2, 2)
net [[0.3775 0.3925]]
out [[0.59326999 0.59688438]]
H1 (1, 2) (2, 2)
net [[1.10590597 1.2249214 ]]
out [[0.75136507 0.77292847]]
Y
 [[1.10590597 1.2249214 ]]

a(Y)
 [[0.75136507 0.77292847]]


In [148]:
### 

## Random initialization

### Softmax Sample and Test

In [147]:
from importlib import reload
reload(network)
model = network.NN(loss='cross_entropy')
model.add_layer(network.Layer(2, 5, 'sigmoid',  label="H1"))
model.add_layer(network.Layer(5, 2, 'sigmoid',  label="output"))
model.show_weights()
X = np.array([[0.05, 0.10]])
Y, aY = model.feed_forward(X)

print("Y\n",Y)
print("\na(Y)\n",aY)

Model Weights
-------------------------------
H1      (input=2, neurons=5, activation=sigmoid)
[[0.15986824 0.78072515 0.15652473 0.39257536 0.409696  ]
 [0.93212521 0.50905525 0.61632734 0.81478519 0.29754673]]
output  (input=5, neurons=2, activation=sigmoid)
[[0.41174386 0.26122794]
 [0.15697582 0.3219758 ]
 [0.11543481 0.6027406 ]
 [0.20306723 0.40751777]
 [0.25848853 0.05099097]]
-------------------------------
H1 (1, 2) (2, 5)
net [[1.10120593 1.08994178 1.06945897 1.10110729 1.05023947]]
out [[0.75048599 0.74837076 0.74449401 0.75046752 0.74082088]]
output (1, 5) (5, 2)
net [[1.85631369 2.22934598]]
out [[0.8648667  0.90285401]]
Y
 [[1.85631369 2.22934598]]

a(Y)
 [[0.8648667  0.90285401]]


## Testing the loss functions 

### Mean Squared Error

In [113]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])
lr = 0.01
eps  = 0.01
max_it = 100
for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.mse_derivative(I, aY, Y_)
    W = W - lr * dC

assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)
    


0 [ 2.7  8.1 16.2]
1 [ 2.48  7.44 14.88]
2 [ 2.282  6.846 13.692]
3 [ 2.1038  6.3114 12.6228]
4 [ 1.94342  5.83026 11.66052]
5 [ 1.799078  5.397234 10.794468]
6 [ 1.6691702  5.0075106 10.0150212]
7 [1.55225318 4.65675954 9.31351908]
8 [1.44702786 4.34108359 8.68216717]
9 [1.35232508 4.05697523 8.11395045]
10 [1.26709257 3.8012777  7.60255541]
11 [1.19038331 3.57114993 7.14229987]
12 [1.12134498 3.36403494 6.72806988]
13 [1.05921048 3.17763145 6.35526289]
14 [1.00328943 3.0098683  6.0197366 ]
15 [0.95296049 2.85888147 5.71776294]
16 [0.90766444 2.72299332 5.44598665]
17 [0.866898   2.60069399 5.20138798]
18 [0.8302082  2.49062459 4.98124919]
19 [0.79718738 2.39156213 4.78312427]
20 [0.76746864 2.30240592 4.60481184]
21 [0.74072178 2.22216533 4.44433066]
22 [0.7166496  2.1499488  4.29989759]
23 [0.69498464 2.08495392 4.16990783]
24 [0.67548617 2.02645852 4.05291705]
25 [0.65793756 1.97381267 3.94762534]
26 [0.6421438  1.9264314  3.85286281]
27 [0.62792942 1.88378826 3.76757653]
28 [0.615

### Cross Entropy 

In [114]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.cross_entropy_derivative(I, aY, Y_)
    W = W - lr * dC

assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)

0 [ 2.7  8.1 16.2]
1 [0.423 1.269 2.538]
2 [0.502695 1.508085 3.01617 ]


### Sum of Squared Errors 

In [115]:
reload(loss_functions)
I = np.array([[1.5], [4.5], [9]])
Y_  = np.array([[0.5], [1.5], [3]])
W = np.array([[1.8]])

for i in range(max_it):
    aY  = I * W
    print(i, aY.flatten())
    if np.absolute((Y_ - aY)).mean() < eps: 
        break
    dC = loss_functions.smd_derivative(I, aY, Y_)
    W = W - lr * dC
assert(i > 0)
assert(np.absolute((Y_ - aY)).mean() < eps)    

0 [ 2.7  8.1 16.2]
1 [0.423 1.269 2.538]
2 [0.502695 1.508085 3.01617 ]
